In [1]:
import pandas as pd
import numpy as np
import torch
src_path = '/Users/Jipeng/PycharmProjects/simulated_multisensory_integration/'
file_name = 'simulated_data.csv'
df = pd.read_csv(src_path+file_name)
print (df)

    Unnamed: 0  elbow_angle  elbow_x  elbow_y  hand_x  hand_y  shoulder_angle
0            0         50.4     1.21     2.75   -0.72    2.20            66.3
1            1         53.4     1.21     2.75   -0.74    2.30            66.3
2            2         56.4     1.21     2.75   -0.76    2.40            66.3
3            3         56.4     1.35     2.68   -0.64    2.44            63.3
4            4         56.4     1.21     2.75   -0.76    2.40            66.3
5            5         56.4     1.06     2.81   -0.89    2.36            69.3
6            6         59.4     1.06     2.81   -0.91    2.46            69.3
7            7         59.4     1.21     2.75   -0.78    2.51            66.3
8            8         59.4     1.35     2.68   -0.65    2.54            63.3
9            9         59.4     1.21     2.75   -0.78    2.51            66.3
10          10         62.4     1.21     2.75   -0.79    2.61            66.3
11          11         59.4     1.21     2.75   -0.78    2.51   

## Goal of the module
Training the proprioceptive inference model. Use the two joints angle data to predict the position of the hand.

### Input
shoulder_angle, elbow_angle

What precision should the input be divided into? It depends on the performance. I will explore later.

### Output
spatial_coding

Depends on  the precision of the final coding, the area can be divided into N parts. So there are N position labels. From [1,0,0,...0] to [0,0,0,...1]

### Model structure
input-->FC-->FC-->Softmax
### To do
1. Let the model run, without considering the performance.<br>
a. Convert the hand position into spatial coding.<br>
b. Split the dataset.<br>
c. Use PyTorch to design the model.

2. Augment the data, let it cover more positions.
3. Divide the data into appropriate precision.

In [2]:
# Test the function, delete later.
print (df['hand_x'])
print (np.around(df['hand_x']))
print (df.loc[:,['elbow_angle','shoulder_angle']])
print (df.shape[0])

0    -0.72
1    -0.74
2    -0.76
3    -0.64
4    -0.76
5    -0.89
6    -0.91
7    -0.78
8    -0.65
9    -0.78
10   -0.79
11   -0.78
12   -0.76
13   -0.64
14   -0.76
15   -0.64
16   -0.65
17   -0.65
18   -0.65
19   -0.65
20   -0.65
21   -0.51
22   -0.65
23   -0.64
24   -0.49
25   -0.48
26   -0.49
27   -0.51
28   -0.36
29   -0.34
30   -0.19
31   -0.04
32   -0.00
33   -0.04
34   -0.06
35   -0.21
36   -0.19
37   -0.16
38   -0.13
39    0.04
40   -0.13
41   -0.09
42   -0.26
43   -0.22
44   -0.26
45   -0.22
46   -0.18
47   -0.35
48   -0.53
49   -0.71
50   -0.88
51   -1.05
52   -1.22
53   -1.38
Name: hand_x, dtype: float64
0    -1.0
1    -1.0
2    -1.0
3    -1.0
4    -1.0
5    -1.0
6    -1.0
7    -1.0
8    -1.0
9    -1.0
10   -1.0
11   -1.0
12   -1.0
13   -1.0
14   -1.0
15   -1.0
16   -1.0
17   -1.0
18   -1.0
19   -1.0
20   -1.0
21   -1.0
22   -1.0
23   -1.0
24   -0.0
25   -0.0
26   -0.0
27   -1.0
28   -0.0
29   -0.0
30   -0.0
31   -0.0
32   -0.0
33   -0.0
34   -0.0
35   -0.0
36   -0.0
37   -0

In [3]:
feature_names = df.columns.tolist()
print feature_names

label = np.around(df['hand_x'])
df['label'] = np.around(df['hand_x'])
df['label'] = df['label'].astype(np.int)
#print (df)

['Unnamed: 0', 'elbow_angle', 'elbow_x', 'elbow_y', 'hand_x', 'hand_y', 'shoulder_angle']


In [4]:
from torch.utils.data import Dataset, DataLoader, sampler

class ArmDataset(Dataset):
    def __init__(self,df):
        self.dataset = df
    def __len__(self):
        return self.dataset.shape[0]
    def __getitem__(self, idx):
        shoulder_angle = self.dataset.loc[idx,'shoulder_angle']
        elbow_angle = self.dataset.loc[idx,'elbow_angle']
        #elbow_angle = self.dataset.loc[idx,'elbow_angle']
        label = self.dataset.loc[idx,'label']
        sample = {'shoulder_angle':shoulder_angle,'elbow_angle':elbow_angle,'label':label}
        return sample

In [10]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(2,20)
        self.fc2 = nn.Linear(20,3)
    def forward(self,x):
        x = F.tanh(self.fc1(x))
        x = self.fc2(x)
        
        return x
    
net = Net()

In [6]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr = 0.001, momentum=0.9)

In [7]:
def train_test_loader(train_ratio,dataset):
    n_data = len(dataset)
    split = int(n_data*train_ratio)
    
    indices = list(range(n_data))
    train_sampler = sampler.SubsetRandomSampler(indices[:split])
    test_sampler = sampler.SubsetRandomSampler(indices[split:])
    
    train_loader = DataLoader(dataset, sampler=train_sampler, shuffle=False, batch_size=1)
    test_loader = DataLoader(dataset, sampler=test_sampler, shuffle=False, batch_size=1)
    
    return train_loader, test_loader

In [11]:
df = df[df['label']>=0]
df = df.reset_index(drop=True)
arm_dataset = ArmDataset(df)
arm_dataloader = DataLoader(arm_dataset,batch_size=4, shuffle=True)
for idx, batch_samples in enumerate(arm_dataloader):
    shoulder_batches, elbow_batches, label_batches = batch_samples['shoulder_angle'],batch_samples['elbow_angle'],batch_samples['label']
    print(shoulder_batches,label_batches)
    

train_loader, test_loader = train_test_loader(0.7,arm_dataset)

for epoch in range(2):
    
    running_loss = 0.0
    for idx, batch_samples in enumerate(train_loader, 0):
        shoulder_batches, elbow_batches, label_batches = batch_samples['shoulder_angle'],batch_samples['elbow_angle'],batch_samples['label']
        # Clears the gradients of all optimized tensors
        optimizer.zero_grad()
        shoulder_batches = shoulder_batches.view(1,-1)
        elbow_batches = elbow_batches.view(1,-1)
        inputs = torch.cat([shoulder_batches,elbow_batches],dim=0).T
        #print (inputs, label_batches)
        outputs = net(inputs)
        loss = criterion(outputs,label_batches)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if idx % 2 == 1:
            print ('[%d, %5d] loss: %.3f'%
                   (epoch+1, idx+1, running_loss/10))
            running_loss = 0.0
    print ('Finished Training')

(tensor([54.3000, 60.3000, 60.3000, 54.3000]), tensor([0, 0, 0, 0]))
(tensor([51.3000, 60.3000, 54.3000, 51.3000]), tensor([0, 0, 0, 0]))
(tensor([57.3000, 54.3000, 51.3000, 57.3000]), tensor([0, 0, 0, 0]))
(tensor([57.3000, 54.3000, 57.3000, 54.3000]), tensor([0, 0, 0, 0]))
(tensor([51.3000, 60.3000, 51.3000, 54.3000]), tensor([0, 0, 0, 0]))
(tensor([57.3000, 57.3000, 57.3000]), tensor([0, 0, 0]))
[1,     2] loss: 0.320
[1,     4] loss: 0.320
[1,     6] loss: 0.320
[1,     8] loss: 0.320
[1,    10] loss: 0.320
[1,    12] loss: 0.320
[1,    14] loss: 0.320
[1,    16] loss: 0.320
Finished Training
[2,     2] loss: 0.320
[2,     4] loss: 0.320
[2,     6] loss: 0.320
[2,     8] loss: 0.320
[2,    10] loss: 0.320
[2,    12] loss: 0.320
[2,    14] loss: 0.320
[2,    16] loss: 0.320
Finished Training
